In [1]:
import math
import random
import csv
import numpy as np
from flask import Flask, request, jsonify, session
import logging
from flask_cors import CORS


In [ ]:
# Initialize the Flask app
app = Flask(__name__)
CORS(app) 
# Set a secret key to use sessions in Flask
app.secret_key = 'your-secret-key'  # Change this to something secret

# Constants for ACO
ALPHA = 1.0
BETA = 5.0
EVAPORATION_RATE = 0.5
Q = 100
NUM_ANTS = 10
NUM_ITERATIONS = 100

# Distance calculation (Haversine formula)
def calculate_distance(node1, node2):
    R = 6371  # Earth's radius in kilometers
    lat1, lon1 = math.radians(node1.latitude), math.radians(node1.longitude)
    lat2, lon2 = math.radians(node2.latitude), math.radians(node2.longitude)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# GraphNode class for geographical info
class GraphNode:
    def __init__(self, id, latitude, longitude):
        self.id = id
        self.latitude = latitude
        self.longitude = longitude

class Vendor(GraphNode):
    def __init__(self, id, name, latitude, longitude):
        super().__init__(id, latitude, longitude)
        self.name = name

class Shop(GraphNode):
    def __init__(self, id, name, latitude, longitude):
        super().__init__(id, latitude, longitude)
        self.name = name

# Ant class representing one ant in the ACO algorithm
class Ant:
    def __init__(self, start_node, nodes, pheromone_matrix, distance_matrix):
        self.start_node = start_node
        self.nodes = nodes
        self.pheromone_matrix = pheromone_matrix
        self.distance_matrix = distance_matrix
        self.visited_nodes = []
        self.current_position = start_node
        self.total_distance = 0.0

    def select_next_node(self):
        unvisited_nodes = [node for node in self.nodes if node not in self.visited_nodes]
        if not unvisited_nodes:
            return None

        probabilities = []
        for node in unvisited_nodes:
            pheromone = self.pheromone_matrix[self.current_position.id][node.id]
            distance = self.distance_matrix[self.current_position.id][node.id]
            prob = (pheromone ** ALPHA) * ((1.0 / distance) ** BETA)
            probabilities.append(prob)

        total_prob = sum(probabilities)
        probabilities = [prob / total_prob for prob in probabilities]

        r = random.uniform(0, 1)
        cumulative_prob = 0.0
        for i, node in enumerate(unvisited_nodes):
            cumulative_prob += probabilities[i]
            if r <= cumulative_prob:
                return node

    def tour(self):
        self.visited_nodes.append(self.current_position)
        while len(self.visited_nodes) < len(self.nodes):
            next_node = self.select_next_node()
            if not next_node:
                break
            self.total_distance += calculate_distance(self.current_position, next_node)
            self.current_position = next_node
            self.visited_nodes.append(next_node)
        self.total_distance += calculate_distance(self.current_position, self.start_node)
        return self.visited_nodes, self.total_distance

# ACO algorithm
def aco_routing(start_node, nodes, num_ants=NUM_ANTS, num_iterations=NUM_ITERATIONS):
    num_nodes = len(nodes) + 1

    pheromone_matrix = np.ones((num_nodes, num_nodes))
    distance_matrix = np.zeros((num_nodes, num_nodes))

    all_nodes = [start_node] + nodes
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            distance = calculate_distance(all_nodes[i], all_nodes[j])
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance

    best_distance = float('inf')
    best_tour = None

    for _ in range(num_iterations):
        ants = [Ant(start_node, nodes, pheromone_matrix, distance_matrix) for _ in range(num_ants)]
        for ant in ants:
            tour, distance = ant.tour()
            if distance < best_distance:
                best_distance = distance
                best_tour = tour

        pheromone_matrix *= (1 - EVAPORATION_RATE)
        for ant in ants:
            for i in range(len(ant.visited_nodes) - 1):
                start_node = ant.visited_nodes[i]
                end_node = ant.visited_nodes[i + 1]
                pheromone_matrix[start_node.id][end_node.id] += Q / ant.total_distance
                pheromone_matrix[end_node.id][start_node.id] += Q / ant.total_distance

    return best_tour, best_distance

# Function to calculate the best route
def get_aco_path(vendors, shops):
    all_nodes = vendors + shops
    start_node = vendors[0]  # Assuming the rider starts at the first vendor
    best_tour, best_distance = aco_routing(start_node, all_nodes[1:])
    return {
        "distance": best_distance,
        "path": [node.name for node in best_tour]
    }

# Generate Google Maps link
def generate_google_maps_link(tour):
    base_url = "https://www.google.com/maps/dir/"
    waypoints = "/".join([f"{node.latitude},{node.longitude}" for node in tour])
    return base_url + waypoints

# CSV parsing
def parse_csv(file, entity_type):
    data = []
    file_content = file.read().decode("utf-8").splitlines()
    reader = csv.DictReader(file_content)

    # Define only the required columns: 'id', 'Latitude', 'Longitude'
    required_columns = ['id', 'Latitude', 'Longitude']

    # Check if the CSV file contains the required columns
    if not all(column in reader.fieldnames for column in required_columns):
        raise ValueError(f"Missing column in the CSV file. Expected columns: {required_columns}")

    # Parse the rows based on the entity type (vendor or shop)
    for row in reader:
        if entity_type == 'vendor' or entity_type == 'shop':
            # No 'Vendor Name' or 'Name' required, just use 'id', 'Latitude', 'Longitude'
            data.append(
                Vendor(int(row['id']), None, float(row['Latitude']), float(row['Longitude']))
                if entity_type == 'vendor' else
                Shop(int(row['id']), None, float(row['Latitude']), float(row['Longitude']))
            )
    return data

route_result = {}

@app.route('/api/upload-csv', methods=['POST'])
def upload_csv():
    try:
        app.logger.info("Received request to /api/upload-csv")
        
        # Check for required files
        if 'vendor_file' not in request.files or 'shop_file' not in request.files:
            app.logger.error("Missing files: vendor_file or shop_file")
            return jsonify({'error': 'Both vendor_file and shop_file are required.'}), 400

        vendor_file = request.files['vendor_file']
        shop_file = request.files['shop_file']
        
        app.logger.info(f"Vendor file: {vendor_file.filename}, Shop file: {shop_file.filename}")

        vendor_file.seek(0)
        shop_file.seek(0)

        # Parse CSV files
        vendors = parse_csv(vendor_file, 'vendor')
        shops = parse_csv(shop_file, 'shop')

        if not vendors or not shops:
            return jsonify({'error': 'Vendors and shops must not be empty.'}), 400

        # Get best path using ACO algorithm
        path = get_aco_path(vendors, shops)

        # Generate the map link
        tour_nodes = vendors + shops
        map_link = generate_google_maps_link(tour_nodes)

        # Log the result to the console for debugging purposes
        app.logger.info(f"Generated route: {path['path']}")
        app.logger.info(f"Map link: {map_link}")

        # Store the result in the global variable (no session required)
        global route_result
        route_result = {
            'message': 'Route calculated successfully!',
            'distance': path['distance'],
            'path': path['path'],
            'map_link': map_link
        }

        # Print the stored result
        print(f"Storing route_result: {route_result}")

        # Return the response
        return jsonify(route_result), 200

    except Exception as e:
        app.logger.error(f"Error occurred: {e}")
        return jsonify({'error': str(e)}), 500


@app.route('/api/get-route', methods=['GET'])
def get_route():
    try:
        # Access the global variable to fetch the result
        global route_result
        
        # Log the session data to check if it exists
        print(f"Retrieved route result: {route_result}")  # Print the stored data

        # Check if the result exists
        if not route_result:
            app.logger.error("No route result found.")
            return jsonify({'error': 'No route calculated yet. Please upload the files first.'}), 400

        # Return the route result as a JSON response
        return jsonify(route_result), 200

    except Exception as e:
        app.logger.error(f"Error occurred in GET request: {e}")
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2024-12-23 17:01:52,795] INFO in 2682949106: Received request to /api/upload-csv
[2024-12-23 17:01:52,802] INFO in 2682949106: Vendor file: vendors.csv, Shop file: orders.csv
C:\Users\xaadk\AppData\Local\Temp\ipykernel_13704\2682949106.py:63: RuntimeWarning: divide by zero encountered in scalar divide
  prob = (pheromone ** ALPHA) * ((1.0 / distance) ** BETA)
C:\Users\xaadk\AppData\Local\Temp\ipykernel_13704\2682949106.py:67: RuntimeWarning: invalid value encountered in scalar divide
  probabilities = [prob / total_prob for prob in probabilities]
[2024-12-23 17:01:52,855] INFO in 2682949106: Generated route: [None]
[2024-12-23 17:01:52,856] INFO in 2682949106: Map link: https://www.google.com/maps/dir/34.200397,73.243432/34.2003779,73.2434781/34.2003732,73.2434759/34.200344,73.2434486/34.2003744,73.243476/34.2003782,73.2434708
127.0.0.1 - - [23/Dec/2024 17:01:52] "POST /api/upload-csv HTTP/1.1" 200 -


Storing route_result: {'message': 'Route calculated successfully!', 'distance': 0.0, 'path': [None], 'map_link': 'https://www.google.com/maps/dir/34.200397,73.243432/34.2003779,73.2434781/34.2003732,73.2434759/34.200344,73.2434486/34.2003744,73.243476/34.2003782,73.2434708'}


127.0.0.1 - - [23/Dec/2024 17:01:57] "GET /api/get-route HTTP/1.1" 200 -


Retrieved route result: {'message': 'Route calculated successfully!', 'distance': 0.0, 'path': [None], 'map_link': 'https://www.google.com/maps/dir/34.200397,73.243432/34.2003779,73.2434781/34.2003732,73.2434759/34.200344,73.2434486/34.2003744,73.243476/34.2003782,73.2434708'}
